In [1]:
import os
import shutil
import random
import cv2
import numpy as np
from tqdm.notebook import trange, tqdm
#creates 2 different train sets one has only non-augmented pictures, the other has non augmented and augmented

augmentLessRoot = 'augmentLessTrain'
augmentedRoot = 'augmentedTrain'
testRoot = 'test'

def resizeImg(img):
    w = 224
    h = 224
    return cv2.resize(img, (w,h), interpolation = cv2.INTER_AREA)

#resets folders
def initFolders():
    classes = ['stop', 'go', 'back', 'left', 'right']
    roots = [augmentLessRoot, augmentedRoot, testRoot]
    for r in roots:
        if os.path.isdir(f'{r}'):
            shutil.rmtree(f'{r}')
        os.mkdir(f'{r}')
        for c in classes:
            os.mkdir(f'{r}/{c}')
            
initFolders()
counter = 0
for c in os.listdir("data"):
    for imgPath in os.listdir(f'data/{c}'):
        imgPathFull = f'data/{c}/{imgPath}'
        roll = random.random()
        if roll > .2 :#80 20 train test split
            shutil.copy(imgPathFull, f'{augmentLessRoot}/{c}/{counter}.png')
            shutil.copy(imgPathFull, f'{augmentedRoot}/{c}/{counter}.png')
        else:
            shutil.copy(imgPathFull, f'{testRoot}/{c}/{counter}.png')
        counter += 1



In [2]:
#Randomly crops the edges (10%) off of images, changing the ratio of the image and relative position
def directionCrop(img):
    toReturn = [img]
    height = img.shape[0]
    width = img.shape[1]
    height10 = int(height * .1)
    width10 = int(width * .1)
    heightStarts = [0, height10]
    heightEnds = [height - height10, height]
    widthStarts = [0, width10]
    widthEnds = [width - width10, width]
    for hStart in heightStarts:
        for hEnd in heightEnds:
            for wStarts in widthStarts:
                for wEnds in widthEnds:
                    roll = random.random()
                    if roll > .6:#40% chance
                        toReturn.append(img[hStart:hEnd, wStarts: wEnds])
    return toReturn

#changes the angle of the image, adding noise the corners where opencv puts black color
def angleChange(img, a):
    whiteImg = np.ones(img.shape)
    height, width = img.shape[:2]
    center = (width/2, height/2)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=1)
    rotated_image = cv2.warpAffine(src=img, M=rotate_matrix, dsize=(width, height))
    #builds noise to fill the black part of the rotated image
    whiteImg =  cv2.warpAffine(src=whiteImg, M=rotate_matrix, dsize=(width, height))
    whiteImg = np.where(whiteImg > 0 , 0, 1)
    whiteImg = whiteImg *  np.random.uniform(0,1, img.shape)
    whiteImg *= 255
    whiteImg = whiteImg.astype('uint8')
    rotated_image += whiteImg
    return rotated_image

#processes an image, returning its augmentations
def preProcess(img):
    toReturn = []
    imgs = directionCrop(img)
    angles = [-3, 0, 3]
    counter = 0
    for originalImage in imgs:
        originalImage = resizeImg(originalImage)
        toReturn.append(originalImage)
        for a in angles:
            if a != 0:
                i = angleChange(originalImage, a)
                toReturn.append(i)
            else:
                i = originalImage
            mean = 0
            var = 30
            sigma = var**0.5
            gauss = np.random.normal(mean,sigma,(224,224,3))
            noisy = i + gauss
            toReturn.append(noisy)
    return toReturn

In [3]:
#Pass a root and a maximum number of augmentations, augments the pictures in the root directory up to maxAugs per label
def augmentFolder(root, maxAugs = None):
    for label in os.listdir(root):
        idx = 0
        imgPaths = os.listdir(f'{root}/{label}')
        augmentedImages = []
        for imgPath in tqdm(imgPaths):
            img = cv2.imread(f'{root}/{label}/{imgPath}', cv2.IMREAD_COLOR)
            os.remove(f'{root}/{label}/{imgPath}')
            resized = resizeImg(img)
            cv2.imwrite(f'{root}/{label}/processed-{idx}.png', resized)
            idx += 1
            processedImgs = preProcess(img)
            for i in processedImgs:
                augmentedImages.append(i)
        if maxAugs == None:
            for img in augmentedImages:
                cv2.imwrite(f'{root}/{label}/processed-{idx}.png', img)
                idx += 1
        else:
            for _ in range(maxAugs):
                randIndex = int(random.random() * len(augmentedImages))
                randImg = augmen tedImages[randIndex]
                cv2.imwrite(f'{root}/{label}/processed-{idx}.png', randImg)
                idx += 1
        

In [4]:
augmentFolder(augmentedRoot, 50)

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

In [5]:
augmentFolder(testRoot)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
#Resizes images in the train set that has no augmentations so it can be passed into the model
for label in os.listdir(augmentLessRoot):
    for img in os.listdir(f'{augmentLessRoot}/{label}'):
        imgPath = f'{augmentLessRoot}/{label}/{img}'
        img = cv2.imread(imgPath, cv2.IMREAD_COLOR)
        img = resizeImg(img)
        cv2.imwrite(imgPath, img)